In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../../data/processed/data.csv')

In [3]:
df.head(3)

,end_of_period,total_assets,non_current_assets,current_assets,property_plant_equipment,intangible_assets,inventories,trade_receivables,cash_and_cash_equivalents,equity_shareholders_of_the_parent,share_capital,retained_earning_accumulated_losses,non_current_liabilities,current_liabilities,non_current_loans_and_borrowings,financial_liabilities_loans_borrowings,total_shares,file_name,ticker,target
0,2011-03-01,1342.0,86.0,1256.0,0.0,0.0,0.0,0.0,0.0,1221.0,0.0,0.0,0.0,110.0,0.0,0.0,1870.76,11BIT.xlsx,11B,7.54
1,2011-06-01,1659.0,103.0,1556.0,0.0,0.0,0.0,0.0,0.0,1595.0,0.0,0.0,0.0,64.0,0.0,0.0,1870.76,11BIT.xlsx,11B,24.00
2,2011-09-01,2141.0,85.0,2055.0,0.0,0.0,0.0,0.0,0.0,2060.0,0.0,0.0,0.0,81.0,0.0,0.0,1870.76,11BIT.xlsx,11B,14.95


max number of rows that can occur: 99

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15495 entries, 0 to 15494
Data columns (total 20 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   end_of_period                           15495 non-null  object 
 1   total_assets                            15495 non-null  float64
 2   non_current_assets                      15495 non-null  float64
 3   current_assets                          15495 non-null  float64
 4   property_plant_equipment                15495 non-null  float64
 5   intangible_assets                       15495 non-null  float64
 6   inventories                             15495 non-null  float64
 7   trade_receivables                       15495 non-null  float64
 8   cash_and_cash_equivalents               15495 non-null  float64
 9   equity_shareholders_of_the_parent       15495 non-null  float64
 10  share_capital                           15495 non-null  fl

In [6]:
ticker_counts = df['ticker'].value_counts()
ticker_counts.to_csv('test.csv', index=False)
